***Sequences assembly-1 (hardcore solution)***

Use permutation to get every possible order of the reads

For each possible order, conduct the function 'overlap' to assemble the order and store the one has shortest length 

In [5]:
def overlap(a, b, min_length=3):
    """ Return length of longest suffix of 'a' matching
        a prefix of 'b' that is at least 'min_length'
        characters long.  If no such overlap exists,
        return 0. """
    start = 0  # start all the way at the left
    while True:
        start = a.find(b[:min_length], start)  # look for b's suffx in a
        if start == -1:  # no more occurrences to right
            return 0
        # found occurrence; check for full suffix/prefix match
        if b.startswith(a[start:]):
            return len(a)-start
        start += 1  # move just past previous match

import itertools

def scs(ss):
    """ Returns shortest common superstring of given
        strings, which must be the same length """
    shortest_sup = None
    sscount = 0
    ss_dict= {}
    for ssperm in itertools.permutations(ss):
        sup = ssperm[0]  # superstring starts as first string
        for i in range(len(ss)-1):
            # overlap adjacent strings A and B in the permutation
            olen = overlap(ssperm[i], ssperm[i+1], min_length=1)
            # add non-overlapping portion of B to superstring
            sup += ssperm[i+1][olen:]
        
        if shortest_sup is None or len(sup) <= len(shortest_sup):
            shortest_sup = sup  # found shorter superstring

            if len(sup) not in ss_dict:
                ss_dict[len(sup)] = set([sup])
            else:
                ss_dict[len(sup)].update([sup])

    return shortest_sup, sscount,ss_dict  # return shortest

In [7]:
qs= ['ABC','BCA','CAB']
qs2= ['CCT','CTT','TGC','TGG','GAT','ATT']
ss, c,d = scs(qs2)
print(d)
print(d[11])
print(len(ss)) #11
print(len(d[11])) #4

{12: {'CCTTGCGATTGG', 'CCTTGCTGGATT'}, 11: {'CCTTGGATTGC', 'GATTGCCTTGG', 'TGCCTTGGATT', 'TGGATTGCCTT'}}
{'CCTTGGATTGC', 'GATTGCCTTGG', 'TGCCTTGGATT', 'TGGATTGCCTT'}
11
4


In [8]:
from google.colab import files
uploaded = files.upload()

Saving ads1_week4_reads.fq to ads1_week4_reads (1).fq


In [4]:
def readFastaq(filename):
    sequences = []
    qualities = []
    with open(filename) as fh:
        while True:
            fh.readline()
            seq = fh.readline().rstrip()
            fh.readline()
            qual = fh.readline().rstrip()
            if len(seq) ==0:
                break
            sequences.append(seq)
            qualities.append(qual)
    return sequences, qualities
seqs, _ = readFastaq('ads1_week4_reads.fq')

In [16]:
def overlap(a, b, min_length=3):
    """ Return length of longest suffix of 'a' matching
        a prefix of 'b' that is at least 'min_length'
        characters long.  If no such overlap exists,
        return 0. """
    start = 0  # start all the way at the left
    while True:
        start = a.find(b[:min_length], start)  # look for b's prefix in a
        if start == -1:  # no more occurrences to right
            return 0
        # found occurrence; check for full suffix/prefix match
        if b.startswith(a[start:]):
            return len(a)-start
        start += 1  # move just past previous match


***Sequence assembly-2 (better solution for big dataset)***

Execute 'overlap' only when the suffix of a read match the prefix of another read

For each read, we first create a dict included every substring of this read. Then, we check if other reads' suffix match any of these substrings. For each hit, we execute 'overlap' and store the hit has shortest overlap length.

Complete this method on every read.

In [17]:
def pick_maximal_overlap(seqs, k):
    """ Return a pair of reads from the list with a
        maximal suffix/prefix overlap >= k.  Returns
        overlap length 0 if there are no such overlaps."""
    kmersD = {} #everytime we need to create this dict again, keep updating when two reads link together
    ks = 6
    for j in seqs:
        for i in range(len(j)-ks+1): # for each read, take out every substrings
            kmer = j[i:i+ks]
            if kmer not in kmersD:  # use substring to create dict, add corresponding read behind
                kmersD[kmer] = set([j])
            else:
                kmersD[kmer].update([j])

    reada, readb = None, None
    best_olen = 0
    for i in seqs:                  # for each read, take its suffix
        query = i[len(i)-6:len(i)]
        if query in kmersD:         # if this suffix appeared in kmer dict, conduct overlap function for every candidates in the kmer dict
           for j in kmersD[query]:
               if i !=j:
                  olen = overlap(i, j, 30)
                  if olen > best_olen:
                     reada, readb = i, j
                     best_olen = olen # throughout entire sheet, the best
    return reada, readb, best_olen

In [18]:
read_a, read_b, olen = pick_maximal_overlap(seqs,30) #completed after 2m37s
while olen>0:
    seqs.remove(read_a)
    seqs.remove(read_b) # when someone is removed, then the dict of suffix-reads need update as well
    seqs.append(read_a + read_b[olen:])
    read_a, read_b, olen = pick_maximal_overlap(seqs, 30)

In [19]:
ans=''.join(seqs)
print(len(ans)) #15894
print(ans.count('A')) #4633
print(ans.count('T')) #3723

15894
4633
3723
